In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [25]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

## Cross Validation

- Split Dataset : Train & Test Set
- Train Test Split : Train Set & Validation Set => A B V
    - Model1 Train AB, Validate w/ C => Score
    - Model1 Train AC, Validate w/ B => Score
    - Model1 Train BC, Validate w/ A => Score
    - Mean Score
    - repeat...

<hr>

### K-Fold Cross Validation

In [26]:
x = [1,2,3,4,5,6,7,8,9]
from sklearn.model_selection import KFold
kf = KFold(n_splits= 9)

In [27]:
for i, j in kf.split(x):
    print (i,j)

[1 2 3 4 5 6 7 8] [0]
[0 2 3 4 5 6 7 8] [1]
[0 1 3 4 5 6 7 8] [2]
[0 1 2 4 5 6 7 8] [3]
[0 1 2 3 5 6 7 8] [4]
[0 1 2 3 4 6 7 8] [5]
[0 1 2 3 4 5 7 8] [6]
[0 1 2 3 4 5 6 8] [7]
[0 1 2 3 4 5 6 7] [8]


In [28]:
kf2 = KFold(n_splits= 3)

for i, j in kf2.split(x):
    print (i, j)

[3 4 5 6 7 8] [0 1 2]
[0 1 2 6 7 8] [3 4 5]
[0 1 2 3 4 5] [6 7 8]


<hr>

In [29]:
from sklearn.datasets import load_iris
iris = load_iris()

In [30]:
df = pd.DataFrame(iris["data"], columns= ["SL", "SW", "PL", "PW"])
df["target"] = iris["target"]
df

,SL,SW,PL,PW,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


#### Splitting Data before Cross Validate

In [31]:
from sklearn.model_selection import train_test_split
xtr, xts, ytr, yts = train_test_split(df[["SL", "SW", "PL", "PW"]], df["target"], test_size = .1)

In [32]:
dfTr = pd.concat([xtr, ytr], axis= 1)
dfTs = pd.concat([xts, yts], axis= 1)

dfTr

,SL,SW,PL,PW,target
106,4.9,2.5,4.5,1.7,2
41,4.5,2.3,1.3,0.3,0
47,4.6,3.2,1.4,0.2,0
103,6.3,2.9,5.6,1.8,2
12,4.8,3.0,1.4,0.1,0
...,...,...,...,...,...
99,5.7,2.8,4.1,1.3,1
124,6.7,3.3,5.7,2.1,2
122,7.7,2.8,6.7,2.0,2
109,7.2,3.6,6.1,2.5,2


In [33]:
kf = KFold(n_splits= 5)
for i, j in kf.split(np.array(dfTr)):
    print (i, j)
    break

[ 27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44
  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62
  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80
  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98
  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116
 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26]


i = index train <br>
j = index validation

In [34]:
def scoring(model, xtr, xts, ytr, yts):
    model.fit(xtr, ytr)
    return model.score(xts, yts)

In [35]:
score = []
for i, j in kf.split(np.array(dfTr)):
    xtrcv = dfTr.iloc[i][["SL", "SW", "PL", "PW"]]
    ytrcv = dfTr.iloc[i]["target"]
    xvalcv = dfTr.iloc[j][["SL", "SW", "PL", "PW"]]
    yvalcv = dfTr.iloc[j]["target"]
    score.append(scoring(LogisticRegression(), 
                        xtrcv, xvalcv, ytrcv, yvalcv))
    
score

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

[0.9629629629629629, 1.0, 1.0, 0.9629629629629629, 0.9259259259259259]

In [36]:
np.mean(score)

0.9703703703703702

#### Cross Validation Score Methods

In [37]:
from sklearn.model_selection import cross_val_score

print(cross_val_score(LogisticRegression(), dfTr[["SL", "SW", "PL", "PW"]], 
                      dfTr["target"], cv= 5))


print(np.mean(cross_val_score(LogisticRegression(), dfTr[["SL", "SW", "PL", "PW"]], 
                      dfTr["target"], cv= 5)))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

[0.96296296 1.         1.         0.96296296 0.92592593]
0.9703703703703702


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

#### Try Scoring Another Model

In [38]:
from sklearn.neighbors import KNeighborsClassifier
model1 = KNeighborsClassifier(n_neighbors= 3)
model2 = KNeighborsClassifier(n_neighbors= 4)

In [39]:
score1 = cross_val_score(model1, dfTr[["SL", "SW", "PL", "PW"]], dfTr["target"], cv= 5, scoring="balanced_accuracy")
score2 = cross_val_score(model2, dfTr[["SL", "SW", "PL", "PW"]], dfTr["target"], cv= 5, scoring="balanced_accuracy")

score1, score2

(array([0.96666667, 1.        , 0.96296296, 0.92962963, 0.93333333]),
 array([0.93333333, 1.        , 0.96296296, 0.92962963, 0.96666667]))

In [40]:
score1.mean(), score2.mean()

(0.9585185185185185, 0.9585185185185183)

<hr>

### Training => Train & Validation:
- K-Fold, Repeated, Stratisfied
- Shuffle Split
- Leave One Out
- Leave P Out